<a href="https://colab.research.google.com/github/vksgm/leetcode/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
#from transformers import AutoModel, AutoTokenizer

#from transformers import pipeline

# As no model is passed, it chooses the default model based on the task name
#whenever calling the pipeline it does
# 1.Tokenization
# 2.Passing the inputs to the model
# 3. Post Processing

classifier = pipeline("sentiment-analysis")
classifier([ "I love AI.",
           " I dont like "])






No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Building the model from scratch by calling the one which we need

**NOTE: THIS MODEL NEEDS TO BE TRAINED AGAIN AS WE CHANGED THE CONFIG &
NOT LOADING THE PRETRAINED MODEL **



In [ ]:
from transformers import BertConfig

# We can also customize the config of the BERT Model
# Create a BertConfig instance with a modified hidden size
custom_config = BertConfig(
    hidden_size=1024,  # Changing the hidden size
    num_hidden_layers=24 # Changing the number of layers
)

# Print the custom config to see the changes
print(custom_config)

# You can also modify an existing config object
default_config = BertConfig()
print("Original hidden size:", default_config.hidden_size)
default_config.hidden_size = 512 # Modifying the hidden size
print("Modified hidden size:", default_config.hidden_size)

# As we know this needs to be trained again.
# so simply we can load the pretrained model of the BERT
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased")

#We can replace the exact model with automodel class also so that
# It picks the model at runtime based on the scenario/task


BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Original hidden size: 768
Modified hidden size: 512


SAVING A MODEL

In [ ]:
#Lets save the model after doing some config change
from transformers import BertConfig, BertModel

# Create a BertConfig instance with a modified hidden size and attention heads
custom_config = BertConfig(
    hidden_size=1024,  # Changing the hidden size
    num_hidden_layers=24, # Changing the number of layers
    num_attention_heads=16 # Changing the number of attention heads to be a divisor of hidden_size
)

# Instantiate the model directly using the custom config
# This creates a model with the defined architecture but randomly initialized weights
model = BertModel(custom_config)

#save the model
model.save_pretrained("/content/sample_data")

print(f"Model with custom config saved to /content/sample_data")

Model with custom config saved to /content/sample_data


**SAVING A TOKENIZER**

In [8]:
import torch
from transformers import BertTokenizer, AutoModel

#load the tokenizer ( we can use automodel tokenizer though along with the checkpoint name )
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

#get the token output with paddings and special tokens
# WE SHOULD USE THIS AS IT PERFORMS ENCODING + PADDING
encoded_input = tokenizer("You are crushing it!", return_tensors="pt")
print(encoded_input)

#save the tokens
tokenizer.save_pretrained("/content/sample_data")

#Internally it does followin 2 main steps
#Note this comprises of 2 steps of  Encoding
#step 1: converting text to tokens
tokens = tokenizer.tokenize("You are crushing it")
print(tokens)

#step 2: converting tokens to numbers (using Vocab)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

#step 3: converting numbers to tensors (This is not needed when using return_tensors="pt")
input_ids = torch.tensor(ids)

#BUT you cannot pass thes tokens directly to model as the dimensions are not as expected
# we need further processing as mentioned above like padding, change dimensions etc

checkpoint = "bert-base-cased"

#model is assigned
model = AutoModel.from_pretrained(checkpoint)

#This wont work as it doesnt match with the expected dimensions of the model
#print('The output without proper dimension',model(input_ids))

# This will now work because encoded_input is a dictionary with tensors that have a batch dimension
print( 'the output with proper encoding',model(**encoded_input))

# DECODING THE TOKENS
decoded_string = tokenizer.decode(ids)
print(decoded_string)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  1192,  1132, 15477,  1122,   106,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
['You', 'are', 'crushing', 'it']
[1192, 1132, 15477, 1122]


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

the output with proper encoding BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4005,  0.2883,  0.1467,  ..., -0.1675,  0.1514, -0.1791],
         [ 0.5877,  0.1216,  0.7655,  ...,  0.3804, -0.0602,  0.3135],
         [ 0.2430,  0.3954,  0.1499,  ...,  0.5597, -0.3013, -0.0577],
         ...,
         [ 0.2784,  0.4924, -0.5341,  ..., -0.1563, -0.3581,  0.0329],
         [ 0.1023, -0.0754,  0.5723,  ...,  0.3628,  0.0698,  0.0085],
         [ 0.5514,  0.8785, -0.0155,  ..., -0.0395,  0.2783, -0.3174]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-6.4373e-01,  4.6753e-01,  9.9987e-01, -9.9278e-01,  9.7728e-01,
          9.1916e-01,  9.8563e-01, -9.8844e-01, -9.7022e-01, -7.1817e-01,
          9.7845e-01,  9.9847e-01, -9.9756e-01, -9.9975e-01,  7.9277e-01,
         -9.7670e-01,  9.8569e-01, -5.3681e-01, -9.9995e-01, -8.1948e-01,
         -5.4771e-01, -9.9982e-01,  2.8432e-01,  9.5082e-01,  9.7269e-01,
          8.2775e-02,  9.8343e-01, 

Lets see what is happening inside the Pipeline for an Automodel to predict Hidden state.

Tokenizer --> Model --> provides hidden state --> Post processing( activation function )


In [ ]:
# Going thorugh the tokenizer
from transformers import AutoModel, AutoTokenizer
#load the pretrained tokenizer
#save the pretrained tokenizer as checkpoint and then load it
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#return as tensors ( pytorch )
raw_inputs = [ "I love AI.",
           " I dont "]
 #padding is for padding zeros for shorter text
 # trunction helps to truncate if the texts are too long than the context window
 #this also adds special tokens like CLS( for classification ) SEP-Seperate Sentences
inputs = tokenizer(raw_inputs, return_tensors="pt", padding=True, truncation=True)
#Only tokens are generated, embedding matrix, postional encoding, attentions etc
# Will be handled when passing this tensor input to the models
print('The input raw text is', raw_inputs)
print('The tokenized conversion of raw input is', inputs)


# Going thorugh the model and pass the input as tensor( tokens ) to the model

#Download and cache the pretrained model
#The AutoModels are used to provide the last hidden states,
#it doesnt provide the raw logits for classification.
#ITS MORE GENERIC TO GET THE OUTPUT EMBEDDINGS
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

#model is assigned
model = AutoModel.from_pretrained(checkpoint)

#UNPACKING the dictionary
outputs = model(**inputs)
#Now we got the output tensors & hiddent state by using the automodel
print('The transformer output as tensors(3 dim )',outputs.last_hidden_state)
print('The output dimension is', outputs.last_hidden_state.shape)

# --The purpose of Automodel is done now---#
#lets see the output logits for a classifier model

#Loading a fine tuned model serving for a specific purpose
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
classi_outputs = model(**inputs)

# the raw, unnormalized scores outputted by the last layer of the model.
print('The logits output for a classifier model', classi_outputs.logits)

#Post Processing the output
#applying the necessary activation function
import torch

predictions = torch.nn.functional.softmax(classi_outputs.logits, dim=-1)
print('Predictions in Numerical Format', predictions)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

The input raw text is ['I love AI.', ' I dont ']
The tokenized conversion of raw input is {'input_ids': tensor([[ 101, 1045, 2293, 9932, 1012,  102],
        [ 101, 1045, 2123, 2102,  102,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0]])}


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

The transformer output as tensors(3 dim ) tensor([[[ 0.5465,  0.2547,  0.1591,  ...,  0.4576,  1.0761, -0.4736],
         [ 0.6439,  0.4412,  0.1479,  ...,  0.5219,  1.0058, -0.3880],
         [ 0.8868,  0.4628,  0.3834,  ...,  0.4210,  1.0010, -0.3718],
         [ 0.3882,  0.3635,  0.1790,  ...,  0.5411,  0.9342, -0.4837],
         [ 0.5676,  0.2252,  0.4042,  ...,  0.5247,  0.9638, -0.7490],
         [ 0.8512,  0.2793,  0.6173,  ...,  0.6684,  0.7761, -0.9155]],

        [[ 0.0543,  0.3361,  0.1553,  ..., -0.1703,  0.0485,  0.6187],
         [-0.0236,  0.5314,  0.0797,  ..., -0.2669,  0.1185,  0.8146],
         [-0.1956,  0.7956,  0.5394,  ..., -0.2056,  0.0273, -0.2130],
         [-0.4307,  0.1253,  0.4421,  ..., -0.2104,  0.2153,  0.5187],
         [ 0.0719,  0.3533, -0.0379,  ...,  0.1257, -0.1557, -0.2085],
         [-0.0706,  0.3748, -0.1036,  ..., -0.5420,  0.2098, -0.0773]]],
       grad_fn=<NativeLayerNormBackward0>)
The output dimension is torch.Size([2, 6, 768])
The logits 

Zero shot classifier

In [ ]:
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification")

# Classify the sentence and get the full result
result = classifier(
    "This is about the Transformers library",
    candidate_labels=["Deep Learning", "technology"],
)

print(result)

# The result will typically be a list containing one dictionary
# The 'scores' list within that dictionary corresponds to the candidate_labels in the same order
scores = result['scores']
labels = result['labels']

# Find the index of the maximum score
max_score_index = scores.index(max(scores))

# Get the label corresponding to the maximum score
max_label = labels[max_score_index]

# You can also get the max score itself
max_score = scores[max_score_index]

# Now you can print or use the max_label and max_score
print(f"The label with the maximum score is: {max_label}")
print(f"The maximum score is: {max_score}")

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'sequence': 'This is about the Transformers library', 'labels': ['technology', 'Deep Learning'], 'scores': [0.9804858565330505, 0.01951419562101364]}
The label with the maximum score is: technology
The maximum score is: 0.9804858565330505


**Text Generator**

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-360M")
generator(
    "my baby will born on",
    max_length=30,
    num_return_sequences=2,
    num_beams=2,  # Set num_beams to enable generating multiple sequences
    do_sample=True # Set do_sample to get diverse outputs
)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'my baby will born on the 1st of July 2019.\n\nI am a single mother. I have a 3'},
 {'generated_text': 'my baby will born on the 1st of July 2019.\n\nI am a single mother. I have a 5'}]

**Named Entity Recognition**

In [ ]:
from transformers import pipeline

ner = pipeline("ner")
ner("A.PMoller HuggingFace Brooklyn")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'entity': 'I-PER',
  'score': np.float32(0.8561442),
  'index': 1,
  'word': 'A',
  'start': 0,
  'end': 1},
 {'entity': 'I-PER',
  'score': np.float32(0.7382918),
  'index': 3,
  'word': 'PM',
  'start': 2,
  'end': 4},
 {'entity': 'I-PER',
  'score': np.float32(0.7276084),
  'index': 4,
  'word': '##oll',
  'start': 4,
  'end': 7},
 {'entity': 'I-ORG',
  'score': np.float32(0.39498562),
  'index': 5,
  'word': '##er',
  'start': 7,
  'end': 9},
 {'entity': 'I-ORG',
  'score': np.float32(0.6500921),
  'index': 8,
  'word': '##F',
  'start': 17,
  'end': 18},
 {'entity': 'I-ORG',
  'score': np.float32(0.62326723),
  'index': 9,
  'word': '##ace',
  'start': 18,
  'end': 21},
 {'entity': 'I-ORG',
  'score': np.float32(0.8617757),
  'index': 10,
  'word': 'Brooklyn',
  'start': 22,
  'end': 30}]

 **ZERO PADDING & ATTENTION MASKS**

In [7]:
import  torch
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
#zero padding to make tensors with equal shape; else it will fail due to dimension issue
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

#here the logits output will be different for the 2nd row ( where padding is done )
# This is because the attention layer is missing :)

#(output( logits in this case ) of the model will be different because attention mask is missing
#print(model(torch.tensor(sequence1_ids)).logits)
#print(model(torch.tensor(sequence2_ids)).logits)
print('Logits Without Attention Masks',model(torch.tensor(batched_ids)).logits)

#lets add the attention mask
#1 consider the elementl; 0 dont consider
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

#Now output will be same for both rows as both has same input tensor values
#but unluckily its not happening: seems we need to also add positional encodings!!
print('Logits with Attention Masks',model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask)))


Logits Without Attention Masks tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)
Logits with Attention Masks SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
